## 제주특산물경진대회 - 이혜승 1차 코드

## Import

In [2]:
import random
import pandas as pd
import numpy as np
import os

from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder

import warnings
warnings.filterwarnings(action='ignore') 

## Fixed Random-Seed

In [3]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42) # Seed 고정

## Load Data

In [4]:
train_df = pd.read_csv('./train.csv')
test_df = pd.read_csv('./test.csv')

## Data Pre-Processing

In [5]:
#시계열 특성을 학습에 반영하기 위해 timestamp를 월, 일, 시간으로 나눕니다
train_df['year'] = train_df['timestamp'].apply(lambda x : int(x[0:4]))
train_df['month'] = train_df['timestamp'].apply(lambda x : int(x[5:7]))
train_df['day'] = train_df['timestamp'].apply(lambda x : int(x[8:10]))

test_df['year'] = test_df['timestamp'].apply(lambda x : int(x[0:4]))
test_df['month'] = test_df['timestamp'].apply(lambda x : int(x[5:7]))
test_df['day'] = test_df['timestamp'].apply(lambda x : int(x[8:10]))

#### 참고) 결측치 test train에 모두 존재 X

TG : 감귤 (Citrus fruits)

BC : 브로콜리 (Broccoli)

RD : 무 (Radish)

CR : 당근 (Carrots)

CB : 양배추 (Cabbage)

In [6]:
international_trade_df = pd.read_csv('./international_trade.csv')
international_trade_df.head()

,기간,품목명,수출 중량,수출 금액,수입 중량,수입 금액,무역수지
0,2019-01,토마토(신선한 것이나 냉장한 것으로 한정한다),356571,990,0,0,990
1,2019-01,양파,821330,222,4003206,1118,-896
2,2019-01,쪽파,60,1,93405,128,-127
3,2019-01,꽃양배추와 브로콜리(broccoli),160,1,638913,563,-562
4,2019-01,방울다다기 양배추,0,0,7580,38,-38


In [7]:
# Translate column names to English
international_trade_df.columns = ['Period', 'Item_Name', 'Export_Weight_kg', 'Export_Value_1000USD', 'Import_Weight_kg', 'Import_Value_1000USD', 'Trade_Balance']

# Mapping of Korean names to item codes based on the user's input
item_name_to_code = {
    '감귤': 'TG',  # Citrus fruits
    '브로콜리': 'BC',  # Broccoli
    '무': 'RD',  # Radish
    '당근': 'CR',  # Carrots
    '양배추': 'CB',  # Cabbage
}

# The international_trade_df contains more items than we need, filter out the irrelevant ones
# First, extract the relevant Korean names for the items we are interested in
relevant_korean_names = list(item_name_to_code.keys())

# Now filter the international_trade_df to only include the relevant items
international_trade_df = international_trade_df[international_trade_df['Item_Name'].str.contains('|'.join(relevant_korean_names))]

# Map the 'Item_Name' to the item codes
international_trade_df['Item_Code'] = international_trade_df['Item_Name'].apply(lambda name: next((code for korean_name, code in item_name_to_code.items() if korean_name in name), None))

# Dropping rows where mapping resulted in None (no matching item code)
international_trade_df = international_trade_df.dropna(subset=['Item_Code'])

# We need to split 'Period' into 'year' and 'month' for aggregation and merging
international_trade_df['year'] = international_trade_df['Period'].apply(lambda x: int(x[0:4]))
international_trade_df['month'] = international_trade_df['Period'].apply(lambda x: int(x[5:7]))

# Now we aggregate the trade data by month and year
international_trade_df_grouped = international_trade_df.groupby(['year', 'month', 'Item_Code']).agg({
    'Export_Weight_kg': 'sum',
    'Export_Value_1000USD': 'sum',
    'Import_Weight_kg': 'sum',
    'Import_Value_1000USD': 'sum',
    'Trade_Balance': 'sum'
}).reset_index()

# Check the transformed and aggregated trade data
international_trade_df_grouped.head(10)


,year,month,Item_Code,Export_Weight_kg,Export_Value_1000USD,Import_Weight_kg,Import_Value_1000USD,Trade_Balance
0,2019,1,BC,160,1,638913,563,-562
1,2019,1,CB,184650,94,403382,128,-34
2,2019,1,CR,23150,22,7466150,2955,-2934
3,2019,1,RD,2627,23,94529,464,-441
4,2019,1,TG,58368,172,0,0,172
5,2019,2,BC,780,1,396870,399,-398
6,2019,2,CB,182636,69,340282,98,-29
7,2019,2,CR,16250,16,6524716,2600,-2585
8,2019,2,RD,0,0,43151,160,-160
9,2019,2,TG,8474,33,0,0,33


In [8]:
# Merge the international trade data with the train and test datasets
train_merged = train_df.merge(international_trade_df_grouped, how='left', left_on=['year', 'month', 'item'], right_on=['year', 'month', 'Item_Code'])

# Drop the 'Item_Code' column as it is redundant after merging
train_merged = train_merged.drop(columns=['Item_Code'])

# Check the first few rows of the merged train dataset and for any missing values introduced by the merge
train_merged.head()


,ID,timestamp,item,corporation,location,supply(kg),price(원/kg),year,month,day,Export_Weight_kg,Export_Value_1000USD,Import_Weight_kg,Import_Value_1000USD,Trade_Balance
0,TG_A_J_20190101,2019-01-01,TG,A,J,0.0,0.0,2019,1,1,58368.0,172.0,0.0,0.0,172.0
1,TG_A_J_20190102,2019-01-02,TG,A,J,0.0,0.0,2019,1,2,58368.0,172.0,0.0,0.0,172.0
2,TG_A_J_20190103,2019-01-03,TG,A,J,60601.0,1728.0,2019,1,3,58368.0,172.0,0.0,0.0,172.0
3,TG_A_J_20190104,2019-01-04,TG,A,J,25000.0,1408.0,2019,1,4,58368.0,172.0,0.0,0.0,172.0
4,TG_A_J_20190105,2019-01-05,TG,A,J,32352.0,1250.0,2019,1,5,58368.0,172.0,0.0,0.0,172.0


In [9]:
# Filter the international trade data to include only the most recent two years
recent_trade_data = international_trade_df_grouped[
    international_trade_df_grouped['year'].isin([2021, 2022])
]

# Calculate the monthly averages for these two years
monthly_averages_recent = recent_trade_data.groupby(['month', 'Item_Code']).mean().reset_index()

# Merge the monthly averages with the test data
test_merged_with_recent_averages = test_df.merge(monthly_averages_recent, how='left', left_on=['month', 'item'], right_on=['month', 'Item_Code'])

# Drop the 'Item_Code' column as it is redundant after merging
test_merged_with_recent_averages = test_merged_with_recent_averages.drop(columns=['Item_Code'])


# Check the first few rows of the merged test dataset and for any missing values introduced by the merge
#test_merged_with_recent_averages.head(), test_merged_with_recent_averages.isnull().sum()
test_merged_with_recent_averages.head()

,ID,timestamp,item,corporation,location,year_x,month,day,year_y,Export_Weight_kg,Export_Value_1000USD,Import_Weight_kg,Import_Value_1000USD,Trade_Balance
0,TG_A_J_20230304,2023-03-04,TG,A,J,2023,3,4,2021.5,12674.5,59.0,7031.5,23.0,36.0
1,TG_A_J_20230305,2023-03-05,TG,A,J,2023,3,5,2021.5,12674.5,59.0,7031.5,23.0,36.0
2,TG_A_J_20230306,2023-03-06,TG,A,J,2023,3,6,2021.5,12674.5,59.0,7031.5,23.0,36.0
3,TG_A_J_20230307,2023-03-07,TG,A,J,2023,3,7,2021.5,12674.5,59.0,7031.5,23.0,36.0
4,TG_A_J_20230308,2023-03-08,TG,A,J,2023,3,8,2021.5,12674.5,59.0,7031.5,23.0,36.0


In [10]:
# 카테고리 변수 원핫인코딩
# One-hot encoding of categorical variables in both train and test datasets
train_encoded = pd.get_dummies(train_merged, columns=['item', 'corporation', 'location'])
test_encoded = pd.get_dummies(test_merged_with_recent_averages, columns=['item', 'corporation', 'location'])

# Check the first few rows of train_encoded to confirm the changes
train_encoded.head()

,ID,timestamp,supply(kg),price(원/kg),year,month,day,Export_Weight_kg,Export_Value_1000USD,Import_Weight_kg,...,item_RD,item_TG,corporation_A,corporation_B,corporation_C,corporation_D,corporation_E,corporation_F,location_J,location_S
0,TG_A_J_20190101,2019-01-01,0.0,0.0,2019,1,1,58368.0,172.0,0.0,...,False,True,True,False,False,False,False,False,True,False
1,TG_A_J_20190102,2019-01-02,0.0,0.0,2019,1,2,58368.0,172.0,0.0,...,False,True,True,False,False,False,False,False,True,False
2,TG_A_J_20190103,2019-01-03,60601.0,1728.0,2019,1,3,58368.0,172.0,0.0,...,False,True,True,False,False,False,False,False,True,False
3,TG_A_J_20190104,2019-01-04,25000.0,1408.0,2019,1,4,58368.0,172.0,0.0,...,False,True,True,False,False,False,False,False,True,False
4,TG_A_J_20190105,2019-01-05,32352.0,1250.0,2019,1,5,58368.0,172.0,0.0,...,False,True,True,False,False,False,False,False,True,False


In [11]:
test_encoded = test_encoded.rename(columns={'year_x': 'year'}) .drop('year_y', axis=1)

In [12]:
from xgboost import XGBRegressor

# Fill missing values with median of each column
train_data_filled = train_encoded.fillna(train_encoded.median())

# Split features and target
X_train = train_data_filled.drop(['ID', 'timestamp', 'price(원/kg)','supply(kg)'], axis=1) # Dropping ID, timestamp as they are not features
y_train = train_data_filled['price(원/kg)']

# Initialize the XGBoost regressor
xgb = XGBRegressor()

# Fit the model to the training data
xgb.fit(X_train, y_train)


test_new = test_encoded.rename(columns={'year_x': 'year'})
# Prepare the test data (apply the same preprocessing as for the training data)
X_test = test_encoded.drop(['ID', 'timestamp'], axis=1)  # We do not drop 'price(원/kg)' because it's not present in the test set

# Predict the prices using the trained model
y_pred = xgb.predict(X_test)

# Add the predictions to the test dataframe
test_encoded['predicted_price(원/kg)'] = y_pred

# Show the first few rows of the test data with the predictions
test_encoded.head()


TypeError: could not convert string to float: 'TG_A_J_20190101'

## Inference

## Submission

In [43]:
submission = pd.read_csv('C:/Users/hyeseung/Desktop/4-2/Kuggle/제주특산물경진대회/sample_submission.csv')
submission

,ID,answer
0,TG_A_J_20230304,0
1,TG_A_J_20230305,0
2,TG_A_J_20230306,0
3,TG_A_J_20230307,0
4,TG_A_J_20230308,0
...,...,...
1087,RD_F_J_20230327,0
1088,RD_F_J_20230328,0
1089,RD_F_J_20230329,0
1090,RD_F_J_20230330,0


In [44]:
submission['answer'] = y_pred
submission

,ID,answer
0,TG_A_J_20230304,4134.407715
1,TG_A_J_20230305,3265.670898
2,TG_A_J_20230306,4572.926758
3,TG_A_J_20230307,4237.362793
4,TG_A_J_20230308,4330.997070
...,...,...
1087,RD_F_J_20230327,651.384644
1088,RD_F_J_20230328,496.224426
1089,RD_F_J_20230329,-66.206932
1090,RD_F_J_20230330,526.953369


In [45]:
submission.to_csv('C:/Users/hyeseung/Desktop/4-2/Kuggle/제주특산물경진대회/submission.csv', index=False)